__NAME:__ __FULLNAME__  
__SECTION:__ __NUMBER__  
__CS 5703: Machine Learning Practices__

# Homework 7: Model Comparisons

## Assignment Overview
Generally, it's helpful to first read through the entire notebook before writing
any code to obtain a sense of the overall program structure before you start coding.  

Follow the TODOs and read through and understand any provided code.  


### Task
For this assignment, you'll be comparing different models after performing holistic
cross validation to find the best parameter sets for various sizes of the training
data. 

For this assignment, we will try to predict shoulder and elbow torque simultaneously,
from the neural activation. 

### Data set
The BMI (Brain Machine Interface) data are stored in a single pickle file; within this file, there
is one dictionary that contains all of the data.  The keys are: 'MI', 
'theta', 'dtheta', 'torque', and 'time'.  Each of these objects are python lists with 20 
numpy matrices; each matrix contains an independent fold of data, with rows representing 
different samples and columns representing different dimensions.  The samples are organized 
contiguously (one sample every 50ms), but there are gaps in the data.
* _MI_ contains the data for 48 neurons.  Each row encodes the number of action potentials for 
each neuron at each of 20 different time bins (so, 48 x 20 = 960 columns).  
* _theta_ contains the angular position of the shoulder (in column 0) and the elbow 
(in column 1) for each sample.  
* _dtheta_ contains the angular velocity of the shoulder (in column 0) and the elbow 
(in column 1) for each sample.  
* _torque_ contains the torque of the shoulder (in column 0) and the elbow (in column  1) for each sample.  
* _time_ contains the actual time stamp of each sample.

A fold is a subset of the available data.  Cutting the data into folds is useful for adjusting training, validation, and test 
sets sizes, and for assessing the generality of a modelling approach.Each fold contains independent time points.


### Objectives
* Understanding hyper-parameter selection using __holistic cross validation__
* Understanding how training set size affects model choices
* Learning to perform model selection using hypothesis testing

### Notes
* Be sure to adequately label all the plots you generate.
* Make sure to save your notebook and results into your own Google Drive

### General References
* [Guide to Jupyter](https://www.datacamp.com/community/tutorials/tutorial-jupyter-notebook)
* [Python Built-in Functions](https://docs.python.org/3/library/functions.html)
* [Python Data Structures](https://docs.python.org/3/tutorial/datastructures.html)
* [Numpy Reference](https://docs.scipy.org/doc/numpy/reference/index.html)
* [Numpy Cheat Sheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf)
* [Summary of matplotlib](https://matplotlib.org/3.1.1/api/pyplot_summary.html)
* [DataCamp: Matplotlib](https://www.datacamp.com/community/tutorials/matplotlib-tutorial-python?utm_source=adwords_ppc&utm_campaignid=1565261270&utm_adgroupid=67750485268&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=1t1&utm_creative=332661264365&utm_targetid=aud-299261629574:dsa-473406587955&utm_loc_interest_ms=&utm_loc_physical_ms=9026223&gclid=CjwKCAjw_uDsBRAMEiwAaFiHa8xhgCsO9wVcuZPGjAyVGTitb_-fxYtkBLkQ4E_GjSCZFVCqYCGkphoCjucQAvD_BwE)
* [Pandas DataFrames](https://urldefense.proofpoint.com/v2/url?u=https-3A__pandas.pydata.org_pandas-2Ddocs_stable_reference_api_pandas.DataFrame.html&d=DwMD-g&c=qKdtBuuu6dQK9MsRUVJ2DPXW6oayO8fu4TfEHS8sGNk&r=9ngmsG8rSmDSS-O0b_V0gP-nN_33Vr52qbY3KXuDY5k&m=mcOOc8D0knaNNmmnTEo_F_WmT4j6_nUSL_yoPmGlLWQ&s=h7hQjqucR7tZyfZXxnoy3iitIr32YlrqiFyPATkW3lw&e=)
* [Sci-kit Learn Linear Models](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model)
* [Sci-kit Learn Ensemble Models](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble)
* [Sci-kit Learn Metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)
* [Sci-kit Learn Model Selection](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection)
* [SciPy Paired t-test for Dependent Samples](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.ttest_rel.html)
* [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test#Dependent_t-test_for_paired_samples)
* [Understanding Paired t-tests](https://machinelearningmastery.com/how-to-code-the-students-t-test-from-scratch-in-python/)
* [Two-tailed Tests](https://www.investopedia.com/terms/t/two-tailed-test.asp)

## Hand-In Procedure
* Execute all cells so they are showing correct results
* Notebook (from Jupyter or Colab):
  + Submit this file (.ipynb) to the Gradescope Notebook HW7 dropbox
* Note: there is no need to submit a PDF file or to submit directly to Canvas

In [ ]:
import pandas as pd
import numpy as np
import pickle as pkl
import scipy.stats as stats
import os
import pathlib, itertools
import time as timelib
import matplotlib.pyplot as plt

from matplotlib import cm
from sklearn.metrics import explained_variance_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

# Default figure parameters
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['font.size'] = 10
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

%matplotlib inline


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# LOAD DATA

In [ ]:
""" PROVIDED
Load the BMI data from all the folds
"""
#fname = '/home/fagg/datasets/bmi/bmi_dataset.pkl'
fname = '/content/drive/MyDrive/MLP_2022/datasets/bmi_dataset.pkl'

with open(fname, 'rb') as f:
  bmi = pkl.load(f)

MI_folds = bmi['MI'] 
theta_folds = bmi['theta']
dtheta_folds = bmi['dtheta']
torque_folds = bmi['torque']
time_folds = bmi['time']

alldata_folds = zip(MI_folds, theta_folds, dtheta_folds, torque_folds, time_folds)

nfolds = len(MI_folds)
nfolds

In [ ]:
""" PROVIDED
Print out the shape of all the data for each fold
"""
for i, (MI, theta, dtheta, torque, time) in enumerate(alldata_folds):
    print("FOLD %2d " % i, MI.shape, theta.shape, 
          dtheta.shape, torque.shape, time.shape)

In [ ]:
""" Provided
For this homework, it actually becomes rather difficult to work with 
individual arrays of different length, because numpy cannot efficiently
concatenate them together  

Here, we construct a single unified array for our data, which we can 
then be indexed into using the folds_idx array. This allows us to efficiently 
index into our data instead of creating
a copy of it each time we want to change the size of the the training set or
our cross-validation rotation.
"""
folds_idx = [None]*nfolds
unified_idx = 0
for i, fold in enumerate(time_folds):
    # creates a list containing indexes from start of fold to end of fold,
    # eg folds_idx[0] = [0,1,...,1192], folds_idx[1] = [1193,...,2296], ...
    # we don't need to store all this (could just store start indexes),
    # but this makes it a lot easier later
    folds_idx[i] = list(range(unified_idx, unified_idx + fold.shape[0]))
    unified_idx += fold.shape[0]

def concat_folds(folds):
    all_folds = np.array(folds[0])
    for f in folds[1:]:
        all_folds = np.append(all_folds, f, axis=0)
    return all_folds

MI = concat_folds(MI_folds) 
theta = concat_folds(theta_folds)
dtheta = concat_folds(dtheta_folds)
torque = concat_folds(torque_folds)
time = concat_folds(time_folds)

print(MI.shape, theta.shape, dtheta.shape, torque.shape, time.shape)

In [ ]:
print([f[0] for f in folds_idx])

# PARAMETER SET LIST

In [ ]:
""" PROVIDED
Construct the Cartesian product of the parameters.

This implementation is similar to what is used by GridSearchCV to create its
Cartesian product of hyper-parameters.

Note that one can also include other hyper-parameters in the list with only a single
option.  This has the effect of simply setting the same hyper-parameter value for each
of resulting hyper-parameter set.

"""
def generate_paramsets(param_lists):
    '''
    Construct the Cartesian product of the parameters
    PARAMS:
        params_lists: dict of lists of values to try for each parameter.
                      keys of the dict are the names of the parameters
                      values are lists of values to try for the 
                      corresponding parameter
    RETURNS: a list of dicts of hyper-parameter sets.  These make up the 
    Cartesian product of the possible hyper-parameters
    '''
    keys, values = zip(*param_lists.items())
    
    # Determines cartesian product of parameter values
    combos = itertools.product(*values)
    
    # Constructs list of dictionaries
    combos_dicts = [dict(zip(keys, vals)) for vals in combos]
    
    return list(combos_dicts)

# PERFORMANCE EVALUTION

In [ ]:
def mse_rmse(trues, preds):
    ''' PROVIDED
    Compute MSE and rMSE for each column separately.
    '''
    mse = np.sum(np.square(trues - preds), axis=0) / trues.shape[0]
    rmse = np.sqrt(mse)
    return mse, rmse


def score_eval(X, y, preds):
    '''
    Compute the  performance scores for the reulsts produced by a model
    PARAMS:
        X: input feature data
        y: true output for X
        preds: predicted output for X
    RETURNS: results as a dictionary of numpy arrays
        mse: mean squared error for each column
        rmse: rMSE for each column
        fvaf: Fraction of Variance Accounted For, best is 1.0
    '''

    mse, rmse = mse_rmse(y, preds)
    evar = explained_variance_score(y, preds)
    
    #  This is a 
    # dictionary of numpy arrays. The numpy arrays are
    # be row vectors, where each element is the result 
    # for a different output, when using multiple regression.
    # The keys of the dictionary are the name of the performance 
    # metric, and the values are the numpy row vectors
    results = {
        'mse'  : np.reshape(mse,  (1, -1)), 
        'rmse' : np.reshape(rmse, (1, -1)), 
        'fvaf' : np.reshape(evar, (1, -1)),  # Fraction of Variance Accounted For
    }
    return results

# CROSS VALIDATION

In [ ]:
"""
PROVIDED

Modified from HW 6
"""
class KFoldHolisticCrossValidation():
    '''
    Cross-validation class. This class will perform cross-validation across for a 
    single hyper-parameter set.
    '''
    def __init__(self, model, eval_func, rotation_skip=1): #opt_metric, 
                #maximize_opt_metric=False, rotation_skip=1):
        '''
        :param model: The Scikit-Learn model to be trained
        :param eval_func': Python function that will be used to evaluate a model
                                parameters: (inputs, desired outputs, model predictions)
        :param rotation_skip: Number of CV rotations for every one rotation to train & evaluate.  
                                Typical is 1 (train and evaluate all rotations), but when we are 
                                debugging, it is helpful to perform a smaller number of train/evaluate
                                cycles
        '''
        # TODO: set the class variables
        self.model = model
        #self.model = #TODO
        self.eval_func = eval_func
        #self.eval_func = #TODO
        self.rotation_skip = rotation_skip

    def perform_cross_validation(self, X, y, folds_idx, trainsize):
        ''' This is where the bulk of the work will be done
        Perform cross validation for a singular train set size and single 
        hyper-parameter set, by evaluating the model's performance over 
        multiple data set rotations all of the same size.

        NOTE: This function assumes the hyper-parameters have already been 
              set in the model
            
        PARAMS:
            X: numpy array containing all of the input data 
               (folds concatenated together)
            y: numpy array containing all of the output data 
               (folds concatenated together)
            folds_idx: list of lists containing the indexes
                       of each element in each fold, eg
                       folds_idx[i] = [start_idx, ... , end_idx]
            trainsize: number of folds to use for training
            
        RETURNS: train, val, and test set results for all rotations of the  
                 data sets and the summary (i.e., the averages over all the 
                 rotations) of the results. 
                 
                 results is a dictionary of dictionaries of r-by-n numpy 
                 arrays, where r is the number of rotations, and n is the 
                 number of outputs from the model.
                 
                 summary is a dict of dictionaries of 1-by-n numpy arrays containing
                 the mean and standard deviation of the metrics in results across
                 all rotations
                 
                 In our BMI dataset, n = 2 (e.g., shoulder torque and elbow torque)

                 General form:
                     results.keys() = ['train', 'val', 'test']

                     results['train'].keys() = ['metric1', 'metric2', ...]
                     
                     results['train']['metric1'] = numpy_array
                     
                     results = 
                     {
                        'train':
                                 {
                                     'mse' : r_by_n_numpy_array,
                                     'rmse': r_by_n_numpy_array, 
                                     ...
                                 },
                        'val'  : {...},
                        'test' : {...}
                     }
                     
                     summary = 
                     {
                        'train':
                                 {
                                     'mse_mean' : 1_by_n_numpy_array,
                                     'mse_std'  : 1_by_n_numpy_array,
                                     'rmse_mean': 1_by_n_numpy_array, 
                                     'rmse_std' : 1_by_n_numpy_array,
                                     ...
                                 },
                        'val'  : {...},
                        'test' : {...}
                     }

                    For example, you can access the MSE results for the 
                    validation set like so:
                        results['val']['mse'] 
                    For example, you can access the summary (i.e. the average 
                    results over all the rotations) for the test set for the
                    rMSE like so:
                        summary['test']['rmse_mean']                
        '''
        
        # Verify that a valid train set size was provided
        nfolds = len(folds_idx)
        if trainsize > nfolds - 2: 
            err_msg = "ERROR: KFoldHolisticCrossValidation.perform_cross_validation() - "
            err_msg += "trainsize (%d) cant be more than nfolds (%d) - 2" % (trainsize, nfolds)
            raise ValueError(err_msg)
        
        # Set up results data structures for each rotation
        results = {'train': None, 'val': None, 'test': None}
        summary = {'train': {}, 'val': {}, 'test': {}}
        
        model = self.model
        evaluate = self.eval_func
        
        # Rotate through different train, val, and test sets
        for rotation in range(0, nfolds, self.rotation_skip):
            (
                Xtrain, ytrain, Xval, yval,  Xtest, ytest
            ) = self.get_data(X, y, folds_idx, nfolds, rotation, trainsize)

            print(ytrain.shape)
            
            # Train model using the training set
            model.fit(Xtrain, ytrain) # make sure warm_start is False

            # Predict with the model for train, val, and test sets
            preds = model.predict(Xtrain)
            preds_val = model.predict(Xval)
            preds_test = model.predict(Xtest)

            # Evaluate the model for each set
            res_train = evaluate(Xtrain, ytrain, preds)
            res_val = evaluate(Xval, yval, preds_val)
            res_test = evaluate(Xtest, ytest, preds_test)

            # Record the train, val, and test set results. These are dicts 
            # of result metrics, returned by the evaluate function

            if results['train'] is None: 
                # First rotation: initialize structures
                results['train'] = res_train
                results['val'] = res_val
                results['test'] = res_test
            else:
                # Other rotations: add results to existing structures
                for metric in res_train.keys():
                    results['train'][metric] = np.append(results['train'][metric], res_train[metric], axis=0)
                    results['val'][metric] = np.append(results['val'][metric], res_val[metric], axis=0)
                    results['test'][metric] = np.append(results['test'][metric], res_test[metric], axis=0)

        # Compute/record mean and standard deviation for the size for each metric
        #  The mean is across the rotations
        for metric in results['train'].keys():
            for stat_set in ['train', 'val', 'test']:
                summary[stat_set][metric+'_mean'] = np.mean(results[stat_set][metric], 
                                                            axis=0).reshape(1, -1)
                summary[stat_set][metric+'_std'] = np.std(results[stat_set][metric], 
                                                          axis=0).reshape(1, -1)

        return results, summary

    def get_data(self, X, y, folds_idx, nfolds, rotation, trainsize):
        '''
        Determines the fold indices for the train, val, and test set given
        the total number of folds, rotation, and training set size.
        Use these fold indices to get the training, validation, and test sets
        from all_xfolds and all_folds
        '''
        # Determine folds to use 
        # (eg fold 1,2,3 for trainsize=3, rotation=1, nfolds=20)
        trainfolds = (np.arange(trainsize) + rotation) % nfolds
        valfold = (nfolds - 2 + rotation) % nfolds
        testfold = (valfold + 1) % nfolds
        
        # Construct a list to serve as an index into X for our training
        # samples. This will contain the index of each sample the training set
        train_idx = []
        for i in trainfolds:
            # the + operator concatenates raw python lists
            train_idx += folds_idx[i] 

        # Construct train set by indexing into X and y with the indices of the
        #  samples that belong to the training set
        Xtrain = X[train_idx]
        ytrain = y[train_idx]
        
        # Construct validation set using the valfold.
        Xval = X[folds_idx[valfold]]
        yval = y[folds_idx[valfold]]

        # Construct test set using the testfold
        Xtest = X[folds_idx[testfold]]
        ytest = y[folds_idx[testfold]]
        
        return Xtrain, ytrain, Xval, yval, Xtest, ytest


In [ ]:
"""
PROVIDED

Modified from HW 6
"""

class CrossValidationGridSearch():
    '''
    This class is responsible for performing a grid trainsizes x paramsets CV experiments.
    For each grid point, N-fold crossvalidation is performed (with potential skips in the
    possible rotations).
    
    '''
    def __init__(self, model, paramsets, eval_func, opt_metric, 
                 opt_metric_scale = 1,
                 maximize_opt_metric=False, trainsizes=[1], 
                 rotation_skip=1,
                ):
        ''' 
        Class instance constructor
        
        :param model: Model to be learned
        :param paramsets: List of dicts.  Every dict contains a set of hyper-parameters for use in
                            one experiment
        :param eval_func: Python function that will be used to evaluate a model
                                parameters: (inputs, desired outputs, model predictions)
        :param opt_metric: Optimization metric to be used
        :param opt_metric_scale: Scale factor for optimization metric.  Used for plotting
        :param maximize_opt_metric: True -> best model has high value for performance metric; 
                                    False -> best model has low value
        :param trainsizes: A list of training set sizes (in terms of number of folds
        :param rotation_skip: Number of CV rotations for every one rotation to train & evaluate.  
                                Typical is 1 (train and evaluate all rotations), but when we are 
                                debugging, it is helpful to perform a smaller number of train/evaluate
                                cycles
        '''
        # set the instance variables
        self.model = model
        self.paramsets = paramsets
        self.trainsizes = trainsizes
        self.eval_func = eval_func
        self.opt_metric = opt_metric + '_mean'
        self.opt_metric_scale = opt_metric_scale
        self.maximize_opt_metric = maximize_opt_metric
        self.rotation_skip = rotation_skip
        
        # Results attributes
        # Full recording of all results for all paramsets, sizes, rotations,
        # and metrics. This is a list of dictionaries for each paramset
        self.results = []
        # Validation summary report of all means and standard deviations for 
        # all metrics, for all paramsets, and sizes. This is a 3D s-by-r-by-p 
        # numpy array. Where s is the number of sizes, r the number of summary 
        # metrics +2, and p is the number of paramsets
        self.report_by_size = None
        # List of the indices of the best paramset for each size
        self.best_param_inds = None
        print("TRAINSIZES:", self.trainsizes, trainsizes)

    def load_checkpoint(self, fname):
        ''' PROVIDED
        Load a checkpoint file into self.results
        
        :param fname: Full name of the file to load the checkpoint from. 
        '''
        if not os.path.exists(fname):
            raise ValueError('File %s does not exist'%fname)
        
        with open(fname, 'rb') as f:
            self.results = pkl.load(f)
            
    def dump_checkpoint(self, fname):
        ''' PROVIDED
        Write the current set of results to a checkpoint file
        
        :param fname: Full name of file to write checkpoint to
        '''
        with open(fname, 'wb') as f:
            pkl.dump(self.results, f)
            
    def reset_results(self):
        ''' PROVIDED
        Reset the current set of results that are stored internally
        '''
        self.results = []
    
    def cross_validation_gridsearch(self, X, y, folds_idx, checkpoint_fname=None):
        ''' PROVIDED
        Perform the grid search with the given data (X, y, folds_idx).  This grid search is
        smart in that if a specific set of hyper-parameters have already been done (encoded in
        checkpoing), then building and evaluating for those hyper-parameters will not be done 
        again.
        
        :param X: Full set of input features
        :param y: Full set of desired outputs
        :param folds_idx: List of row indices in X/y, one for each fold
        :param checkpoint_fname: Name of the output checkpoint file.  If None, then not written
                to file.
        '''

        cross_val = KFoldHolisticCrossValidation(
            self.model, self.eval_func, 
            rotation_skip = self.rotation_skip
        )        

        # Iterate over the parameter sets
        for params in self.paramsets:

            # Check that we haven't already done this before (from our checkpoint)
            if params in [r['params'] for r in self.results]:
                print('already evaled:', params)
                continue
      
            print('evaling on:', params)
        
            # Set up the results for these parametrs
            param_results = []
            param_summary = None

            # Set the parameters in the model
            self.model.set_params(**params)
            
            # Iterate over the different train set sizes
            # Running cross-validation on thm
            for size in self.trainsizes:
                # Perform Cross-Validation
                result, summary = cross_val.perform_cross_validation(
                    X, y, folds_idx, size
                )
                # Append results in param_results
                param_results.append(result)
                
                # Append the mean and standard deviation statistics (summary)
                if param_summary is None: 
                    param_summary = summary
                else:
                    # For each metric measured, append the summary results
                    for metric in summary['train'].keys():
                        for stat_set in ['train', 'val', 'test']:
                            param_summary[stat_set][metric] = np.append(
                                param_summary[stat_set][metric], 
                                summary[stat_set][metric], 
                                axis=0
                            )
                            
            # Add this param's results to this accumulating results
            self.results.append({
                'params' :params,
                'results':param_results, 
                'summary':param_summary
            })

            # Write the checkpoint file
            if checkpoint_fname is not None:
                self.dump_checkpoint(checkpoint_fname)
        

    def get_reports_all(self):
        ''' PROVIDED
        Generate reports on the internally stored results
        
        :return: Dictionary containing two keys: 'report_by_size', 'best_param_inds'
        '''
        self.report_by_size = self.get_reports()
        self.best_param_inds = self.get_best_params(
            self.opt_metric, self.maximize_opt_metric
        )

        return {
            'report_by_size' : self.report_by_size, 
            'best_param_inds': self.best_param_inds
        }
    
    # Report generation code. Provided, but you should still read it
    """ PROVIDED
    Functions to generate a report of the result of the cross-validation
    """
    def get_reports(self):
        ''' PROVIDED
        Get the mean validation summary of all the parameters for each size
        for all metrics. This is used to determine the best parameter set  
        for each size
        
        RETURNS: the report_by_size as a 3D s-by-r-by-p array. Where s is 
                 the number of train sizes tried, r is the number of summary  
                 metrics evaluated+2, and p is the number of parameter sets.
        '''
        results = self.results
        sizes = np.reshape(self.trainsizes, (1, -1))
        nsizes = sizes.shape[1]
        nparams = len(results)
        
        # Set up the reports objects
        metrics = list(results[0]['summary']['val'].keys())
        colnames = ['params', 'size'] + metrics 
        report_by_size = np.empty((nsizes, len(colnames), nparams), dtype=object)

        # Determine mean val for each paramset for each size for all metrics
        for p, paramset_result in enumerate(results):
            params = paramset_result['params']
            res_val = paramset_result['summary']['val']

            # Compute mean val result for each train size for each metric
            means_by_size = [np.mean(res_val[metric], axis=1) 
                             for metric in metrics]
            
            print("MEAN", means_by_size, len(means_by_size))
            print("SIZES", sizes, len(sizes))
            # Include the train set sizes into the report
            means_by_size = np.append(sizes, means_by_size, axis=0)
            print("SIZES", sizes)
            print("MEAN2", means_by_size)
            # Include the parameter sets into the report
            param_strgs = np.reshape([str(params)]*nsizes, (1, -1))
            means_by_size = np.append(param_strgs, means_by_size, axis=0).T
            print("MEAN3", means_by_size)
            # Append the parameter set means into the report 
            report_by_size[:,:,p] = means_by_size
        return report_by_size

    def get_best_params(self, opt_metric, maximize_opt_metric):
        ''' PROVIDED (Do read through all the provided code)
        Determines the best parameter set for each train size,  
        based on a specific metric.
        
        PARAMS:
            opt_metric: optimized metric. one of the metrics returned 
                        from eval_func, with '_mean' appended for the
                        summary stat. This is the mean metric used to  
                        determine the best parameter set for each size
                        
            maximize_opt_metric: True if the max of opt_metric should be
                                 used to determine the best parameters.
                                 False if the min should be used.
        RETURNS: list of best parameter set indicies for each size 
        '''
        results = self.results
        report_by_size = self.report_by_size 
                
        metrics = list(results[0]['summary']['val'].keys())
        
        # Determine best params for each size, for the optimized metric
        best_param_inds = None
        metric_idx = metrics.index(opt_metric)
        
        # Report info for all paramsets for the optimized metric
        report_opt_metric = report_by_size[:, metric_idx+2, :]
        
        if maximize_opt_metric:
            # Add two for the additional cols for params and size
            best_param_inds = np.argmax(report_opt_metric, axis=1)
        else: 
            best_param_inds = np.argmin(report_opt_metric, axis=1)
        # Return list of best params indices for each size
        return best_param_inds
    
    def get_best_params_strings(self):
        ''' PROVIDED
        Generates a list of strings of the best params for each size
        RETURNS: list of strings of the best params for each size
        '''
        best_param_inds = self.best_param_inds
        results = self.results
        return [str(results[p]['params']) for p in best_param_inds]

    def get_report_best_params_for_size(self, size):
        ''' PROVIDED
        Get the mean validation summary for the best parameter set 
        for a specific size for all metrics.
        PARAMS:
            size: index of desired train set size for the best  
                  paramset to come from. Size here is the index in 
                  the trainsizes list, NOT the actual number of folds.
        RETURNS: the best parameter report for the size as an s-by-m  
                 dataframe. Where each row is for a different size, and 
                 each column is for a different summary metric.
        '''
        best_param_inds = self.best_param_inds
        report_by_size = self.report_by_size 
        
        bp_index = best_param_inds[size]
        size_len = len(size) if type(size) is list else 1
                
        metrics = list(self.results[0]['summary']['val'].keys())
        colnames = ['params', 'size'] + metrics
        report_best_params_for_size = pd.DataFrame(
            report_by_size[size_idx].T[bp_index].reshape(size_len,-1),
            columns=colnames
        )
        return report_best_params_for_size

    """ PROVIDED
    Plotting code to display the result of the grid search and cross-validation
    """

    def plot_cv(self, foldsindices, results, summary, metrics, size, metric_scales=None):
        ''' PROVIDED
        Plotting function for after perform_cross_validation(), 
        displaying the train and val set performances for each rotation 
        of the training set. 
        
        PARAMS:
            foldsindices: indices of the train sets tried
            results: results from perform_cross_validation()
            summary: mean and standard deviations of the results
            metrics: list of result metrics to plot. Available metrics 
                     are the keys in the dict returned by eval_func
            size: train set size
            metric_scales: vector of scale factors for the metrics 
                (must be one per metric)
            
        RETURNS: the figure and axes handles
        '''
        nmetrics = len(metrics)
        if metric_scales is None:
            metric_scales = [1] * nmetrics

        # Initialize figure plots
        fig, axs = plt.subplots(nmetrics, 1, figsize=(12,6))
        fig.subplots_adjust(hspace=.4)
        # When 1 metric is provided, allow the axs to be iterable
        axs = np.array(axs).ravel()

        # Construct each subplot
        for metric, ax, scale in zip(metrics, axs, metric_scales):
            # Compute the mean for multiple outputs
            res_train = np.mean(results['train'][metric], axis=1)
            res_val = np.mean(results['val'][metric], axis=1)
            #res_test = np.mean(results['test'][metric], axis=1)
            # Plot
            ax.plot(foldsindices, scale*res_train, label='train')
            ax.plot(foldsindices, scale*res_val, label='val')
            ax.set(ylabel=metric)
        axs[0].legend(loc='upper right')
        axs[0].set(xlabel='Fold Index')
        axs[0].set(title='Performance for Train Set Size ' + str(size))
        return fig, axs

    def plot_param_train_val(self, metrics, paramidx=0, view_test=False, metric_scales=None):
        ''' PROVIDED
        Plotting function for after grid_cross_validation(), 
        displaying the mean (summary) train and val set performances 
        for each train set size.
        
        PARAMS:
            metrics: list of summary metrics to plot. '_mean' or '_std'
                     must be append to the end of the base metric name. 
                     These base metric names are the keys in the dict 
                     returned by eval_func
            paramidx: parameter set index
            view_test: flag to view the test set results
            metric_scales: vector of scale factors for the metrics 
                (must be one per metric)
                
        RETURNS: the figure and axes handles
        '''
        sizes = self.trainsizes
        results = self.results

        summary = results[paramidx]['summary']
        params = results[paramidx]['params']
        
        nmetrics = len(metrics)
        if metric_scales is None:
            metric_scales = [1] * nmetrics

        # Initialize figure plots
        fig, axs = plt.subplots(nmetrics, 1, figsize=(12,6))
        #fig.subplots_adjust(hspace=.4)
        # When 1 metric is provided, allow the axs to be iterable
        axs = np.array(axs).ravel()

        # Construct each subplot
        for metric, ax, scale in zip(metrics, axs, metric_scales):
            # Compute the mean for multiple outputs
            res_train = np.mean(summary['train'][metric], axis=1)
            res_val = np.mean(summary['val'][metric], axis=1)
            # Plot
            ax.plot(sizes, scale*res_train, label='train')
            ax.plot(sizes, scale*res_val, label='val')
            if view_test:
                res_test = np.mean(summary['test'][metric], axis=1)
                ax.plot(sizes, res_test, label='test')
            ax.set(ylabel=metric)
            ax.set_xticks(sizes)
        axs[-1].set(xlabel='Train Set Size (# of folds)')
        axs[0].set(title=str(params))
        axs[0].legend(loc='upper right')
        return fig, axs
    
    def plot_allparams_val(self, metrics, metric_scales=None):
        ''' PROVIDED
        Plotting function for after grid_cross_validation(), displaying  
        mean (summary) validation set performances for each train size 
        for all parameter sets for the specified metrics.
        
        PARAMS:
            metrics: list of summary metrics to plot. '_mean' or '_std' 
                     must be append to the end of the base metric name. 
                     These base metric names are the keys in the dict 
                     returned by eval_func
            metric_scales: vector of scale factors for the metrics 
                (must be one per metric)
                
        RETURNS: the figure and axes handles
        '''
        sizes = self.trainsizes
        results = self.results
        
        nmetrics = len(metrics)
        if metric_scales is None:
            metric_scales = [1] * nmetrics

        # Initialize figure plots
        fig, axs = plt.subplots(nmetrics, 1, figsize=(10,6))
        
        # When 1 metric is provided, allow the axs to be iterable
        axs = np.array(axs).ravel()

        # Construct each subplot
        for metric, ax, scale in zip(metrics, axs, metric_scales):
            for p, param_results in enumerate(results):
                summary = param_results['summary']
                params = param_results['params']
                # Compute the mean for multiple outputs
                res_val = np.mean(summary['val'][metric], axis=1)                
                ax.plot(sizes, scale*res_val, label=str(params))
            ax.set(ylabel=metric)
            ax.set_xticks(sizes)
        axs[-1].set(xlabel='Train Set Size (# of folds)')
        axs[0].set(title='Validation Performance')
        axs[0].legend(bbox_to_anchor=(1.02, 1), loc='upper left',
                      ncol=1, borderaxespad=0., prop={'size': 8})
        return fig, axs

    def plot_best_params_by_size(self):
        ''' PROVIDED
        Plotting function for after grid_cross_validation(), displaying 
        mean (summary) train and validation set performances for the best 
        parameter set for each train size for the optimized metric.
                     
        RETURNS: the figure and axes handles
        '''
        results = self.results
        metric = self.opt_metric
        best_param_inds = self.best_param_inds
        sizes = np.array(self.trainsizes)

        # Unique set of best params for the legend
        unique_param_sets = np.unique(best_param_inds)
        lgnd_params = [self.paramsets[p] for p in unique_param_sets]

        # Initialize figure
        fig, axs = plt.subplots(2, 1, figsize=(10,6))
        #fig.subplots_adjust(hspace=.4)
        # When 1 metric is provided, allow the axs to be iterable
        axs = np.array(axs).ravel()
        set_names = ['train', 'val']

        # Construct each subplot
        for i, (ax, set_name) in enumerate(zip(axs, set_names)):
            for p in unique_param_sets:
                # Obtain indices of sizes this paramset was best for
                param_size_inds = np.where(best_param_inds == p)[0]
                param_sizes = sizes[param_size_inds]
                param_sizes.sort()
                # Compute the mean over multiple outputs for each size
                param_summary = results[p]['summary'][set_name]
                metric_scores = np.mean(param_summary[metric][param_size_inds,:], axis=1)
                # Plot the param results for each size it was the best for
                ax.scatter(param_sizes, self.opt_metric_scale * metric_scores, s=120, marker=(p+2, 1))
                ax.set_xticks(param_sizes)
                #ax.grid(True)

            set_name += ' Set Performance'
            ax.set(ylabel=metric, title=set_name)

        axs[-1].set(xlabel='Train Set Size (# of folds)')
        axs[0].legend(lgnd_params, bbox_to_anchor=(1.02, 1), loc='upper left',
                      ncol=1, borderaxespad=0., prop={'size': 8})
        return fig, axs


 

# PERFORM CROSS VALIDATION
Initialize holistic cross validation objects to explore Linear, Ridge, Lasso, and ElasticNet models.

The inputs for the models are the MI data and the outputs are the torque (you'll provide the shoulder and elbow simulataneouly, as done in the previous HW).

In [ ]:
""" TODO
Holistic Cross Validation Options:
* ridge_alphas: list of alphas to try for the RIDGE model
* lasso_alphas: list of alphas to try for the LASSO model
* en_alphas: list of alphas to try for the ELASTICNET model
* l1_ratios: list of l1_ratios to try for the ELASTICNET model

* trainsizes: list of number of folds to utilize in the train set
* opt_metric: the optimized metric, returned by the eval_func, used 
  to select the best parameter sets
* maximize_opt_metric: True if the opt_metric is maximized; False 
  otherwise
* skip: the number of folds to skip when rotating through train sets 
  of the same size
"""
ridge_alphas = [1, 10, 100, 1000, 10000] 
lasso_alphas = [.00001, .0001, .001, .01, .1] 
en_alphas = [.0001, .001, .01, .1, 0.2, 0.4] 
l1_ratios = [.001, .01, .1, 0.9] 

trainsizes = [1, 2, 3, 5, 8, 13, 18]  #range(1, nfolds-1)
opt_metric = 'rmse'
maximize_opt_metric = False
# TODO: set skip to 1 for your final results
skip = 4

# True to always run cross validation, false to re-load existing run
# or run cross validation for the first time
force = False 

# Tag for the filename to save the experiments to
#prefix = "/content/drive/MyDrive/Colab Notebooks/hw7_skip%d"%(skip) 
prefix = "./hw7_skip%d"%(skip) 

## [LINEAR REGRESSION](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

Ordinary least squares Linear Regression.

In [ ]:
""" PROVIDED
LinearRegression

Execute cross validation procedure for all sizes for the 
LinearRegression model using cross_validation_gridsearch().
The parameter list for the LinearRegression model is a
list with just an empty dictionary [{}]
"""
time = timelib.time()
# where to save the cross-validation results.
# By default this will be in your Colab Notebook folder,
# if you wish to save it elsewhere replace this with the full paths
lnr_fullcvfname = prefix + "_linear_crossval.pkl"

#lnr_checkpoint_fun = save_checkpoint_generator(lnr_fullcvfname)

# TODO: create the model and cross val grid search objects
model = LinearRegression()

lnr_crossval = CrossValidationGridSearch(
    model, [{}], score_eval, 
    opt_metric, maximize_opt_metric=maximize_opt_metric,
    trainsizes=trainsizes, rotation_skip=skip,
    opt_metric_scale = 180/np.pi
)

# Should we load the checkpoint file?
if not force and os.path.exists(lnr_fullcvfname):
    checkpoint = lnr_crossval.load_checkpoint(lnr_fullcvfname)

# Use grid_cross_validation() to run the full cross 
#       validation procedure
# Note: when testing, run this using small lists of parameters 
#       (e.g. of length 2 or 4) and/or small trainsize lists 
#       (e.g. [1, 2, 3, 4, 5])
# Note: for the final submission, make sure to use the complete 
#       parameter set list and trainsize list provided/specified
#       This will take some time (longer than an hour)
    
lnr_crossval.cross_validation_gridsearch(
        MI, torque, folds_idx,
        lnr_fullcvfname 
    )
lnr_crossval_report = lnr_crossval.get_reports_all()
    
lnr_crossval_report.keys()
    
# Report 
print("Elapsed Time: %.2f min" % ((timelib.time() - time) / 60))
lnr_crossval.model, lnr_crossval.rotation_skip, lnr_crossval.trainsizes

## [RIDGE](https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression)

$\min_w ||y - w^TX||^2_2 + \alpha ||w||^2_2$

$\alpha$: amount of $L_2$ regularization to apply. Larger $\alpha$ greater penalize the model for larger weights

$w$: the weights from the model

$X$: feature or input data

$y$: true outputs


In [ ]:
""" TODO
RIDGE

Initialize a GridSearchCrossValidation object that uses RIDGE
as the model, and the provided r_allparamsets

Execute cross validation procedure for all sizes for the Ridge
model using cross_validation_gridsearch()
"""
time = timelib.time()
r_fullcvfname = prefix + "_ridge_crossval.pkl"

r_param_lists = {'alpha':ridge_alphas}
r_allparamsets = generate_paramsets(r_param_lists)
#print(pd.DataFrame(r_allparamsets))

# TODO: Initialize a GridSearchCrossValidation object using Ridge
model = Ridge()
r_crossval = # TODO

# Report
print("Elapsed Time: %.2f min" % ((timelib.time() - time) / 60))
r_crossval.model, r_crossval.rotation_skip, r_crossval.trainsizes

## [LASSO](https://scikit-learn.org/stable/modules/linear_model.html#lasso)

$\min_w \frac{1}{2 N} ||y - w^TX||^2_2 + \alpha ||w||_1$

$N$: the number of samples


In [ ]:
""" TODO
LASSO

Initialize a GridSearchCrossValidation object that uses LASSO
as the model, and the provided l_allparamsets

Execute cross validation procedure for all sizes for the Lasso
model using cross_validation_gridsearch()
"""
time = timelib.time()

l_fullcvfname = prefix + "_lasso_crossval.pkl"

l_param_lists = {'alpha':lasso_alphas, 'max_iter':[int(1e4)]}
l_allparamsets = #TODO

# Report
print("Elapsed Time: %.2f min" % ((timelib.time() -time) / 60))
l_crossval.model, l_crossval.rotation_skip, l_crossval.trainsizes

## [ELASTICNET](https://scikit-learn.org/stable/modules/linear_model.html#elastic-net)

$\min_w \frac{1}{2 N} ||y - w^TX||^2_2 + \alpha L_1 ||w||_1 + \frac{1}{2} \alpha (1 - L_1) ||w||^2_2$

$\alpha$: Regularization parameter

$L_1$: the $L_1$ ratio

In [ ]:
""" TODO
ELASTICNET

Initialize a GridSearchCrossValidation object that uses ELASTICNET
as the model, and the provided allparamsets

Execute cross validation procedure for all sizes for the ELASTICNET
model using cross_validation_gridsearch)

Re-load the existing experiment
"""
time = timelib.time()
en_fullcvfname = prefix + "_crossval.pkl"

en_param_lists = {'alpha':en_alphas, 'l1_ratio':l1_ratios, 'max_iter':[int(1e4)]}
en_allparamsets = generate_paramsets(en_param_lists)

model = ElasticNet()
en_crossval =  #TODO
# Report
print("Elapsed Time: %.2f min" % ((timelib.time() - time) / 60))

crossval.model, crossval.rotation_skip, crossval.trainsizes

# RESULTS

### Explore the result output structure

In [ ]:
""" PROVIDED
List GridSearchCrossValidation Attributes
"""
dir(en_crossval)

In [ ]:
""" PROVIDED
Results attribute is a list of dictionaries. Each element, or dictionary
corresponds to the results for a single parameter set
"""
len(en_crossval.results), en_crossval.results[0].keys()

In [ ]:
""" PROVIDED
* crossval.results[0]['results'] is a list of dictionaries with the results
  for each size for the parameter set at index 0
* crossval.results[1]['summary'] is a dictionary of summary results for the 
  train, val, and test sets for the parameter set at index 1
"""
len(en_crossval.results[0]['results']), en_crossval.results[1]['summary'].keys()

In [ ]:
""" PROVIDED
* crossval.results[0]['results'][2] is a dictionary with the results
  for the train size at index 2 for the parameter set at index 0
* crossval.results[1]['summary']['val'] is a dictionary of summary (over the 
  sizes) results for the val set for the parameter set at index 1, for all 
  metrics
"""
en_crossval.results[0]['results'][2].keys(), en_crossval.results[1]['summary']['val'].keys()

In [ ]:
""" PROVIDED
* crossval.results[0]['results'][2]['train'] is a dictionary of all results 
  for the train set for the parameter set at index 0, the size at index 2, for 
  all metrics
* crossval.results[1]['summary']['val']['mse_mean'] is a numpy array of 
  averages for the val set for the parameter set at index 1, for the mse. The
  averages are computed over the sizes
"""
en_crossval.results[0]['results'][2]['train'].keys(), en_crossval.results[1]['summary']['val']['mse_mean'].shape

In [ ]:
""" PROVIDED
* crossval.results[0]['results'][2]['train']['mse'] is a dictionary of all 
  results for the train set for the parameter set at index 0, the size at 
  index 2, for the mse, for all rotations (there are 20 rotations when skip=1)
"""
en_crossval.results[0]['results'][2]['train']['mse'].shape

### Best Parameters for Each Size

In [ ]:
""" PROVIDED
Results options:
* size_idx: index of the size from the list of train sizes to examine results
* metrics: list of summary (average) metrics to examine results
"""
# index 3 corresponds to train size 5
size_idx = 3
metrics = ['rmse_mean', 'fvaf_mean']
metric_scales = [180/np.pi, 1]


In [ ]:
""" PROVIDED
Display the lists of the best parameter sets for each size for all
the models, expect the Linear model (as it has only one parameter set)
"""
print("Best Parameter Sets For Each Train Set Size")

print("RIDGE")
r_best_param_info = pd.DataFrame(
    (
        r_crossval.trainsizes, 
        r_crossval.best_param_inds, 
        r_crossval.get_best_params_strings()
    ),
    index=['train_size','param_index','paramset']
)
print(r_best_param_info.T)


print("\nLASSO")
l_best_param_info = pd.DataFrame(
    (
        l_crossval.trainsizes, 
        l_crossval.best_param_inds, 
        l_crossval.get_best_params_strings()
    ),
    index=['train_size','param_index','paramset']
)
print(l_best_param_info.T)

print("\nELASTICNET")
best_param_info = pd.DataFrame(
    (
        en_crossval.trainsizes, 
        en_crossval.best_param_inds, 
        en_crossval.get_best_params_strings()
    ),
    index=['train_size', 'param_index', 'paramset']
)
print(best_param_info.T)

### Plot Best Parameters for Each Size

In [ ]:
""" PROVIDED
LINEAR REGRESSION
Plot the mean (summary) train and validation set performances for 
each train size for the optimized metric. Use plot_best_params_by_size()

Note: for LinearRegression, there is only one parameter set.
"""
lnr_crossval.plot_best_params_by_size() 


In [ ]:
""" TODO
RIDGE
Plot the mean (summary) train and validation set performances for 
the best parameter set for each train size for the optimized
metrics. Use plot_best_params_by_size()
"""


In [ ]:
""" TODO
LASSO
Plot the mean (summary) train and validation set performances for 
the best parameter set for each train size for the optimized
metrics. Use plot_best_params_by_size()
"""


In [ ]:
""" TODO
ELASTICNET
Plot the mean (summary) train and validation set performances for 
the best parameter set for each train size for the optimized
metrics. Use plot_best_params_by_size()
"""


### Plot Performance over the Parameter Space

In [ ]:
'''
PROVIDED
'''
def plot_param_val_for_size(crossval, metric, alphas, sizeidx=0, 
                            metric_scale=1, xlog=False):
    ''' 
    Plotting function for after cross_validation_gridsearch(), 
    displaying the mean (summary) train and val set performances 
    for each alpha, given the size, for RIDGE and LASSO only

    PARAMS:
        crossval: cross validation object
        metric: summary metric to plot. '_mean' or '_std' must be 
                append to the end of the base metric name. These 
                base metric names are the keys in the dict returned
                by eval_func
        alphas: list of alpha values
        sizeidx: train size index
        metric_scale: scale to multiply the metric by before plotting
        xlog: Use log scale along the x axis

    RETURNS: the figure and axes handles
    '''
    sizes = crossval.trainsizes
    results = crossval.results
    best_param_inds = crossval.best_param_inds

    nalphas = len(alphas)

    nsizes = len(sizes)

    # Initialize the matrices for the curve
    Y_train = np.empty((nalphas,))
    Y_val = np.empty((nalphas,))

    # Obtain the mean performance for the curve
    for param_res in results:
        params = param_res['params']
        summary = param_res['summary']

        alpha_idx = alphas.index(params['alpha'])

        # Compute the mean for multiple outputs
        res_train = np.mean(summary['train'][metric][sizeidx, :])
        Y_train[alpha_idx] = res_train

        res_val = np.mean(summary['val'][metric][sizeidx, :])
        Y_val[alpha_idx] = res_val
    
    # Initialize figure plots
    fig = plt.figure(figsize=(12,2))
    for i, (Y, set_name) in enumerate(zip((Y_train, Y_val), 
                                          ('Training', 'Validation'))):
        # Plot
        ax = fig.add_subplot(1, 2, i+1)
        print(alphas)
        ax.plot(alphas, metric_scale * Y)
        ax.set_xticks(alphas)
        title = "%s Performance, Train Size %d Folds" % (set_name, 
                                                         sizes[sizeidx])
        ax.set(title=title)
        ax.set(xlabel="alpha", ylabel=metric)
        if xlog:
            ax.set_xscale('log')
    return fig


In [ ]:
'''
PROVIDED
'''
def plot_surface(xlist, ylist, Z_train, Z_val, 
                 xlabel, ylabel, zlabel, 
                 elev=30, angle=45, title_suffix="",
                xticks=None, yticks=None,
                xlog=False, ylog=False,
                figsize=(10,5),
                tick_decimals=None):
    ''' 
    Helper plotting function. x-axis is always alpha
    
    REQUIRES: from mpl_toolkits.mplot3d import Axes3D

    PARAMS:
        xlist: list of x values
        ylist: list of y values
        Z_train: matrix of performance results from the training set
        Z_val: matrix of performance results from the validation set
        ylabel: y-axis label 
        zlabel: z-axis label
        elev: elevation of the 3D plot for the view
        angle: angle in degrees of the 3D plot for the view
        title_suffix: string to append to each subplot title
        xticks: specify x tick locations
        yticks: specify y tick locations
        xlog: Use log scale for x axis
        ylog: Use log scale for y axis
        figsize: Size of the figure
        tick_decimals: If specified, number of digits after the decimal point

    RETURNS: the figure
    '''
    # Initialize figure
    fig = plt.figure(figsize=figsize)
    
    # Create the X/Y coordinates for the grid
    X, Y = np.meshgrid(xlist, ylist) 
    
    # Use log of X in plot ?
    if xlog:
        X = np.log10(X)
        xticks = np.log10(xticks)
        
    # Use log of Y in plot ?
    if ylog:
        Y = np.log10(Y)
        yticks = np.log10(yticks)
        
    # Plot Training and Validation performance
    for i, (Z, set_name) in enumerate(zip((Z_train, Z_val), 
                                          ('Training', 'Validation'), 
                                              )):
        # Plot the surface
        ax = fig.add_subplot(1,2, i+1, projection='3d')
        surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, 
                               linewidth=0, antialiased=False)
        title = "%s Performance %s" % (set_name, title_suffix)
        ax.view_init(elev=elev, azim=angle)
        ax.set(title=title)
        ax.set(xlabel=xlabel, ylabel=ylabel, zlabel=zlabel)
        
        # Set tick marks
        if xticks is not None:
            tmp = xticks
            
            # Round the ticks if requested
            if tick_decimals is not None:
                tmp = np.round(xticks, decimals=tick_decimals)
            ax.set_xticks(tmp)
            
        # Round the ticks if requested
        if yticks is not None:
            tmp = yticks
            if tick_decimals is not None:
                tmp = np.round(yticks, decimals=tick_decimals)
            ax.set_yticks(tmp)
            
    return fig

In [ ]:
'''
PROVIDED
'''
def plot_param_val_surface_RL(crossval, metric, alphas, metric_scale=1,
                              elev=30, angle=245):
    ''' 
    Plotting function for after cross_validation_gridsearch(), 
    displaying the mean (summary) train and val set performances 
    for each alpha, for all sizes, for RIDGE and LASSO only
    
    REQUIRES: from mpl_toolkits.mplot3d import Axes3D

    PARAMS:
        crossval: cross validation object
        metric: summary metric to plot. '_mean' or '_std' must be 
                append to the end of the base metric name. These 
                base metric names are the keys in the dict returned
                by eval_func
        alphas: list of alpha values
        elev: elevation of the 3D plot for the view
        angle: angle in degrees of the 3D plot for the view

    RETURNS: the figure and axes handles
    '''
    sizes = crossval.trainsizes
    results = crossval.results
    best_param_inds = crossval.best_param_inds
    nalphas = len(alphas)

    nsizes = len(sizes)

    # Initialize the matrices for the surface
    Z_train = np.empty((nsizes, nalphas))
    Z_val = np.empty((nsizes, nalphas))

    # Obtain the mean performance for the surface
    for param_res in results:
        params = param_res['params']
        summary = param_res['summary']

        alpha_idx = alphas.index(params['alpha'])

        # Compute the mean for multiple outputs
        res_train = np.mean(summary['train'][metric], axis=1)
        Z_train[:, alpha_idx] = res_train

        # Compute the mean for multiple outputs
        res_val = np.mean(summary['val'][metric], axis=1)
        Z_val[:, alpha_idx] = res_val
    
    fig = plot_surface(alphas, 
                       sizes, 
                       Z_train*metric_scale, 
                       Z_val*metric_scale, 
                       'log alpha',
                       'size (# of folds)', 
                       metric, elev, angle,
                      xticks=alphas, 
                       yticks=sizes,
                      xlog=True,
                      tick_decimals=1)
    return fig

In [ ]:
'''
PROVIDED
'''
def plot_param_val_surface_EN(crossval, metric, param_lists, 
                              metric_scale=1,
                              sizeidx=0, elev=35, angle=280):
    ''' 
    Plotting function for after cross_validation_gridsearch(), 
    displaying the mean (summary) train and val set performances 
    for each alpha and l1_ratio, given the size, for the ELASTICNET
    
    REQUIRES: from mpl_toolkits.mplot3d import Axes3D

    PARAMS:
        crossval: cross validation object
        metric: summary metric to plot. '_mean' or '_std' must be 
                append to the end of the base metric name. These 
                base metric names are the keys in the dict returned
                by eval_func
        param_lists: dictionary of the list of alphas and l1_ratios
        sizeidx: train size index
        elev: elevation of the 3D plot for the view
        angle: angle in degrees of the 3D plot for the view
        metric_scale: scale factor to apply to metric before plotting

    RETURNS: the figure 
    '''
    sizes = crossval.trainsizes
    results = crossval.results
    best_param_inds = crossval.best_param_inds

    alphas = list(param_lists['alpha'])
    l1_ratios = list(param_lists['l1_ratio'])

    nalphas = len(alphas)
    nl1_ratios = len(l1_ratios)

    nsizes = len(sizes)

    # Initialize the matrices for the surface
    Z_train = np.empty((nl1_ratios, nalphas))
    Z_val = np.empty((nl1_ratios, nalphas))

    # Obtain the mean performance for the surface 
    for param_res in results:
        params = param_res['params']
        summary = param_res['summary']

        alpha_idx = alphas.index(params['alpha'])
        l1_idx = l1_ratios.index(params['l1_ratio'])

        # Compute the mean for multiple outputs
        res_train = np.mean(summary['train'][metric][sizeidx, :])
        Z_train[l1_idx, alpha_idx] = res_train

        res_val = np.mean(summary['val'][metric][sizeidx, :])
        Z_val[l1_idx, alpha_idx] = res_val
    
    fig = plot_surface(alphas, l1_ratios, 
                       Z_train*metric_scale, 
                       Z_val*metric_scale, 
                       'log alpha',
                       'log l1_ratio', 
                       metric, elev, angle,
                       ', Size %d Folds'% sizes[sizeidx],
                      xticks=alphas,
                      yticks=l1_ratios,
                      xlog=True,
                      ylog=True,
                      figsize=(8,4),
                      tick_decimals=1)
    return fig

In [ ]:
""" PROVIDED
List the parameter sets explored for RIDGE
"""
r_crossval.paramsets

In [ ]:
""" PROVIDED
RIDGE
Use plot_param_val_surface_RL() to plot the surface of the training
and validation set performance versus alpha and size in the X and Y axes,
using the optimized metric
"""
# Feel free to adjust these to understand the shape of the surface
# Elevation of the plot
elev = 30
# Angle the plot is viewed
angle = 255

# Plot
plot_param_val_surface_RL(r_crossval, r_crossval.opt_metric, 
                          ridge_alphas, elev=elev, angle=angle,
                         metric_scale=r_crossval.opt_metric_scale)

# UNCLEAR WHY WE ARE GETTING TWO COPIES OF EACH PLOT

In [ ]:
""" PROVIDED
List the parameter sets explored for LASSO
"""
l_crossval.paramsets

In [ ]:
""" TODO
LASSO
Use plot_param_val_surface_RL() to plot the surface of the training
and validation set performance versus alpha and size in the X and Y axes,
using the optimized metric
"""
# Feel free to adjust these to understand the shape of the surface
# Elevation of the plot
elev = 30
# Angle the plot is viewed
angle = 255

# TODO: Plot
plot_param_val_surface_RL(# TODO)

In [ ]:
""" PROVIDED
List the parameter sets explored for ELASTICNET
"""
en_crossval.paramsets

In [ ]:
""" PROVIDED
ELASTICNET
Use plot_param_val_surface_EN() to plot the surface of the training
and validation set performance versus alpha and l1_ratio in the X 
and Y axes for the size indices of 0, 3, and 6, for crossval.opt_metric
"""
# Feel free to adjust these to understand the shape of the surface
# Elevation of the plot
elev = 25
# Angle the plot is viewed
angle = 300

# Plot
for si in [0, 2, 5]:
    plot_param_val_surface_EN(en_crossval, en_crossval.opt_metric, 
                              param_lists=en_param_lists, sizeidx=si, 
                              elev=elev, angle=angle,
                             metric_scale=r_crossval.opt_metric_scale)

### Paired t-tests
We can use paired t-tests to assess statistically significant differences between the mean test set performances of the models

In [ ]:
""" PROVIDED
Obtain all the results for all the models
"""
# LinearRegression
lnr_all_results = lnr_crossval.results

# RIDGE
r_all_results = r_crossval.results

# LASSO
l_all_results = l_crossval.results

# ELASTICNET
all_results = en_crossval.results

In [ ]:
""" PROVIDED

Plot distributions of the Validation and Test scores from the
best parameter set for each base model for the corresponding 
size indices, [0, 3, 5]. The metric of interest is rmse.
These are the distribution of results from each rotation of 
the training set
"""
metric = 'rmse'
set_names = ['val', 'test']
scale = 180.0/np.pi

# Size indices
size_indices = [0, 3, 5]

for si in size_indices:
    # Obtain the index of the best parameter set for the size
    # RIDGE
    r_bp_idx = r_crossval.best_param_inds[si]
    # LASSO
    l_bp_idx = l_crossval.best_param_inds[si]
    # ELASTICNET
    bp_idx = en_crossval.best_param_inds[si]

    # Construct the figure
    fig, axs = plt.subplots(1, 2, figsize=(15,5))
    for i, set_name in enumerate(set_names):
        title = '%s, Size %d folds' % (set_name, trainsizes[si])

        # LINEAR
        # Note: there's only 1 parameter set for the Linear model
        lnr_res = lnr_all_results[0]['results'][si][set_name]
        lnr_scores = np.mean(lnr_res[metric], axis=1) * scale

        # RIDGE
        # Obtain results for the best parameter set for the size
        ridge_res = r_all_results[r_bp_idx]['results'][si][set_name]
        # Compute the mean of the outputs for each data set rotation
        ridge_scores = np.mean(ridge_res[metric], axis=1) * scale

        # LASSO
        lasso_res = l_all_results[l_bp_idx]['results'][si][set_name]
        lasso_scores = np.mean(lasso_res[metric], axis=1) * scale
        
        # ELASTICNET
        res = all_results[bp_idx]['results'][si][set_name]
        elastic_scores = np.mean(res[metric], axis=1) * scale
        
        # Determine the edges for the bins in the histograms
        all_scores = np.concatenate((elastic_scores, ridge_scores, 
                                     lasso_scores, lnr_scores))
    
        
        # Boxplots
        axs[i].boxplot([elastic_scores, ridge_scores, 
                           lasso_scores, lnr_scores])
        axs[i].set_xticklabels(['ElasticNet', 'Ridge', 'Lasso', 'Linear'])
        axs[i].set(ylabel=metric)
        axs[i].set(title=title, xlabel=metric)


In [ ]:
""" TODO
Paired t-test
Two-sided t-test for the null hypothesis that mean of the distribution
of differences between the two test performance distributions is zero 
"""
size_idx = 0

print("Train Set Size", trainsizes[size_idx])

# LINEAR
# Note: there's only 1 parameter set for the LinearRegression model
lnr_res = lnr_crossval.results[0]['results'][size_idx]['test']
lnr_test_res = np.mean(lnr_res[metric], axis=1)

# RIDGE
# Obtain index of best parameters for specified train size
r_bp_idx = r_crossval.best_param_inds[size_idx]
# Obtain all results for the best parameter set specified train size
ridge_res = # TODO
# Compute the mean of the outputs for each data set rotation
ridge_test_res = # TODO

# LASSO
l_bp_idx = # TODO
lasso_res = # TODO
lasso_test_res = # TODO

# TODO: ELASTICNET
bp_idx = # TODO
net_res = # TODO
elastic_test_res = # TODO

In [ ]:
""" PROVIDED
ELASTICNET vs RIDGE
Execute the paired t-test to determine whether to reject the null hypothesis 
(i.e. H0) with 95% confidence. H0 is that the mean of the distribution of the 
differences between test scores for the best ELASTICNET model and the best 
RIDGE is zero, when using a training size of 8 (i.e. the size at index 7 of 
the trainsizes list). Display the t-statistic, the p-value, and the mean of 
the differences (i.e. mean(elastic_test_res - ridge_test_res))

Use stats.ttest_rel(). See the API reference above.
Do the same for all the pairing of models
"""
stats.ttest_rel(elastic_test_res, ridge_test_res, nan_policy='omit')

In [ ]:
# PROVIDED
mean_of_diffs = np.mean(elastic_test_res - ridge_test_res) 
mean_of_diffs * scale

In [ ]:
""" TODO
ELASTICNET vs LASSO
Execute the paired t-test
"""


In [ ]:
# TODO: Compute the difference between elastic net and lasso
mean_of_diffs = 

In [ ]:
""" TODO
ELASTICNET vs LinearRegression
Execute the paired t-test
"""


In [ ]:
# TODO: compute the difference between elastic net and the pure linear model
mean_of_diffs = 

# DISCUSSION
For each question write one brief paragraph of discussion:

1. Interpret the meaning of the t-test results using 95% confidence for the one training fold case. Discuss the statistical meaning, as well as the practical interpretation of the results in the context of the data set.

## TODO

2. Interpret the meaning of the t-test results using 95% confidence for the five training fold case.

## TODO

3. For the Elastic Net Model, discuss the differences in the surfaces between the train sizes of 1, 3, and 13 folds, for both the training and validation sets.

# TODO

4. For each of the train set sizes of 1, 2, 13, 18 folds, which model (Linear, Lasso, Ridge, or ElasticNet) and corresponding parameter set would you select and why? Specify which model and parameter set for each size.  Remember, selections should be made based on the validation performance.

## TODO
